# Deep learning A-Z : Building a Boltzmann Machine

<p align="justify">
This notebook is my response to the fifth homework of the course called *Deep Learning A-Z™: Hands-On Artificial Neural Networks* accessible here : https://www.udemy.com/deeplearning/
</p>
<p align="justify">
In this notebook, we are going to build two recommended systems which will tell for each user if this user liked or no a movie. The first recommended system will have a binary output to tell if the user liked or not and the second one will ouput a rating from 1 to five. We are going to build this recommended system using Boltzmann machines built with pytorch.
</p>

### Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Data preprocessing

In [4]:
path_movies = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__')))), 
                          'ressources/Boltzmann_Machines/ml-1m/movies.dat')
path_users = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__')))), 
                          'ressources/Boltzmann_Machines/ml-1m/users.dat')
path_ratings = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__')))), 
                          'ressources/Boltzmann_Machines/ml-1m/ratings.dat')

movies = pd.read_csv(path_train, sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv(path_users, sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv(path_ratings, sep='::', header=None, engine='python', encoding='latin-1')

movies.head()


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


For the ratings dataset, the first col is the user id, the second is the genre and the third one is the grade. Fourth one is the zip code.

In [5]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


For the ratings dataset, the first col is the user id, the second is the movie that this user rated and the third one is the grade. Fourth one is just a timestamp.

In [6]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2. Building the Boltzmann Machine